In [1]:
import numpy as np
import pandas as pd
import re
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
import json
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv(r'E:\Mumbai Flat Real Estate Intelligence\datasets\MMR_Housing_Residential_Projects.csv')

In [3]:
df.head()

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Lodha Bellissimo,"2, 3 BHK Apartment in Worli, Mumbai","['Worli Sea Face', 'Haji Ali Dargah', 'PD Hind...","{'Sea Face': '500 m', 'Haji Ali': '2 km', 'Ban...",https://www.99acres.com/lodha-bellissimo-worli...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Club House', 'Gymnasium', '..."
1,Oberoi Sky City,"3, 4 BHK Apartment in Worli, Mumbai","['Worli Sea Face', 'Nehru Science Centre', 'Wo...","{'Worli Sea Face': '800 m', 'Bandra-Worli Sea ...",https://www.99acres.com/oberoi-sky-city-worli,"{'3 BHK': {'building_type': 'Apartment', 'area...","['Infinity Pool', 'Concierge Service', 'Privat..."
2,Godrej Platinum,"2, 3 BHK Apartment in Vikhroli, Mumbai","['Godrej Industries', 'IIT Bombay', 'Hirananda...","{'IIT Bombay': '3 km', 'Eastern Express Highwa...",https://www.99acres.com/godrej-platinum-vikhroli,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Gym', 'Club House', 'Joggin..."
3,Hiranandani Gardens,"2, 3, 4 BHK Apartment in Powai, Mumbai","['Powai Lake', 'IIT Bombay', 'Hiranandani Hosp...","{'Powai Lake': '500 m', 'IIT Bombay': '2 km', ...",https://www.99acres.com/hiranandani-gardens-powai,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Tennis Court', 'Squash Cour..."
4,Raheja Vivarea,"1, 2 BHK Apartment in Santacruz East, Mumbai","['Mumbai Airport', 'Bandra Kurla Complex', 'Mi...","{'Mumbai Airport': '3 km', 'BKC': '4 km', 'San...",https://www.99acres.com/raheja-vivarea-santacruz,"{'1 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Gym', 'Club House', 'Childr..."


In [4]:
df.iloc[2].NearbyLocations

"['Godrej Industries', 'IIT Bombay', 'Hiranandani Hospital', 'Kanjurmarg Station', 'Eastern Express Highway']"

In [5]:
df.iloc[2].LocationAdvantages

"{'IIT Bombay': '3 km', 'Eastern Express Highway': '500 m', 'Powai Lake': '4 km', 'Mumbai Airport': '8 km', 'Kanjurmarg Station': '2 km'}"

In [6]:
df.iloc[1].PriceDetails

"{'3 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '2,100 sq.ft.', 'price-range': '₹ 14 - 16 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '2,850 sq.ft.', 'price-range': '₹ 19 - 22 Cr'}}"

In [7]:
df.iloc[2].TopFacilities

"['Swimming Pool', 'Gym', 'Club House', 'Jogging Track', 'Indoor Games', 'Amphitheatre', 'Badminton Court', '24x7 Security', 'Rain Water Harvesting']"

In [8]:
df[['PropertyName','TopFacilities']]['TopFacilities'][0]

'[\'Swimming Pool\', \'Club House\', \'Gymnasium\', \'Landscaped Gardens\', \'24x7 Security\', \'Indoor Games\', \'Jogging Track\', "Children\'s Play Area", \'Party Hall\']'

In [9]:
def extract_list(s):
    return re.findall(r"'(.*?)'", s)

df['TopFacilities'] = df['TopFacilities'].apply(extract_list)

In [10]:
df.head()

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Lodha Bellissimo,"2, 3 BHK Apartment in Worli, Mumbai","['Worli Sea Face', 'Haji Ali Dargah', 'PD Hind...","{'Sea Face': '500 m', 'Haji Ali': '2 km', 'Ban...",https://www.99acres.com/lodha-bellissimo-worli...,"{'2 BHK': {'building_type': 'Apartment', 'area...","[Swimming Pool, Club House, Gymnasium, Landsca..."
1,Oberoi Sky City,"3, 4 BHK Apartment in Worli, Mumbai","['Worli Sea Face', 'Nehru Science Centre', 'Wo...","{'Worli Sea Face': '800 m', 'Bandra-Worli Sea ...",https://www.99acres.com/oberoi-sky-city-worli,"{'3 BHK': {'building_type': 'Apartment', 'area...","[Infinity Pool, Concierge Service, Private The..."
2,Godrej Platinum,"2, 3 BHK Apartment in Vikhroli, Mumbai","['Godrej Industries', 'IIT Bombay', 'Hirananda...","{'IIT Bombay': '3 km', 'Eastern Express Highwa...",https://www.99acres.com/godrej-platinum-vikhroli,"{'2 BHK': {'building_type': 'Apartment', 'area...","[Swimming Pool, Gym, Club House, Jogging Track..."
3,Hiranandani Gardens,"2, 3, 4 BHK Apartment in Powai, Mumbai","['Powai Lake', 'IIT Bombay', 'Hiranandani Hosp...","{'Powai Lake': '500 m', 'IIT Bombay': '2 km', ...",https://www.99acres.com/hiranandani-gardens-powai,"{'2 BHK': {'building_type': 'Apartment', 'area...","[Swimming Pool, Tennis Court, Squash Court, Gy..."
4,Raheja Vivarea,"1, 2 BHK Apartment in Santacruz East, Mumbai","['Mumbai Airport', 'Bandra Kurla Complex', 'Mi...","{'Mumbai Airport': '3 km', 'BKC': '4 km', 'San...",https://www.99acres.com/raheja-vivarea-santacruz,"{'1 BHK': {'building_type': 'Apartment', 'area...","[Swimming Pool, Gym, Club House, Children, , ,..."


In [11]:
df['FacilitiesStr'] = df['TopFacilities'].apply(' '.join)

In [12]:
df['FacilitiesStr'][0]

'Swimming Pool Club House Gymnasium Landscaped Gardens 24x7 Security Indoor Games Jogging Track s Play Area", '

In [13]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))

In [14]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df['FacilitiesStr'])

In [15]:
tfidf_matrix.toarray()[0]

array([0.14680005, 0.14680005, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.28753793, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.05443571, 0.05443571, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.09606404, 0.        , 0.        , 0.12039285, 0.        ,
       0.        , 0.        , 0.        , 0.17858403, 0.28753793,
       0.        , 0.        , 0.        , 0.17858403, 0.        ,
       0.28753793, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.05443571, 0.        , 0.        , 0.        ,
       0.28753793, 0.        , 0.        , 0.        , 0.     

In [16]:
cosine_sim1 = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [17]:
cosine_sim1.shape

(149, 149)

In [18]:
df[['PropertyName','PriceDetails']]['PriceDetails'][1]

"{'3 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '2,100 sq.ft.', 'price-range': '₹ 14 - 16 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '2,850 sq.ft.', 'price-range': '₹ 19 - 22 Cr'}}"

In [19]:
def recommend_properties(property_name, cosine_sim=cosine_sim1):
    # Get the index of the property that matches the name
    idx = df.index[df['PropertyName'] == property_name].tolist()[0]

    # Get the pairwise similarity scores with that property
    sim_scores = list(enumerate(cosine_sim1[idx]))

    # Sort the properties based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar properties
    sim_scores = sim_scores[1:6]

    # Get the property indices
    property_indices = [i[0] for i in sim_scores]
    
    recommendations_df = pd.DataFrame({
        'PropertyName': df['PropertyName'].iloc[property_indices],
        'SimilarityScore': sim_scores
    })

    # Return the top 10 most similar properties
    return recommendations_df

In [20]:
recommend_properties("Piramal Aranya Phase 5")

,PropertyName,SimilarityScore
16,Piramal Revanta,"(16, 0.9999999999999999)"
26,Piramal Heights,"(26, 0.9999999999999999)"
36,Piramal Aranya Phase 2,"(36, 0.9999999999999999)"
46,Piramal Aranya Phase 3,"(46, 0.9999999999999999)"
56,Piramal Aranya Phase 4,"(56, 0.9999999999999999)"


In [21]:


# Function to parse and extract the required features from the PriceDetails column
def refined_parse_modified_v2(detail_str):
    try:
        details = json.loads(detail_str.replace("'", "\""))
    except:
        return {}

    extracted = {}
    for bhk, detail in details.items():
        # Extract building type
        extracted[f'building type_{bhk}'] = detail.get('building_type')

        # Parsing area details
        area = detail.get('area', '')
        area_parts = area.split('-')
        if len(area_parts) == 1:
            try:
                value = float(area_parts[0].replace(',', '').replace(' sq.ft.', '').strip())
                extracted[f'area low {bhk}'] = value
                extracted[f'area high {bhk}'] = value
            except:
                extracted[f'area low {bhk}'] = None
                extracted[f'area high {bhk}'] = None
        elif len(area_parts) == 2:
            try:
                extracted[f'area low {bhk}'] = float(area_parts[0].replace(',', '').replace(' sq.ft.', '').strip())
                extracted[f'area high {bhk}'] = float(area_parts[1].replace(',', '').replace(' sq.ft.', '').strip())
            except:
                extracted[f'area low {bhk}'] = None
                extracted[f'area high {bhk}'] = None

        # Parsing price details
        price_range = detail.get('price-range', '')
        price_parts = price_range.split('-')
        if len(price_parts) == 2:
            try:
                extracted[f'price low {bhk}'] = float(price_parts[0].replace('₹', '').replace(' Cr', '').replace(' L', '').strip())
                extracted[f'price high {bhk}'] = float(price_parts[1].replace('₹', '').replace(' Cr', '').replace(' L', '').strip())
                if 'L' in price_parts[0]:
                    extracted[f'price low {bhk}'] /= 100
                if 'L' in price_parts[1]:
                    extracted[f'price high {bhk}'] /= 100
            except:
                extracted[f'price low {bhk}'] = None
                extracted[f'price high {bhk}'] = None

    return extracted
# Apply the refined parsing and generate the new DataFrame structure
data_refined = []

for _, row in df.iterrows():
    features = refined_parse_modified_v2(row['PriceDetails'])
    
    # Construct a new row for the transformed dataframe
    new_row = {'PropertyName': row['PropertyName']}
    
    # Populate the new row with extracted features
    for config in ['1 BHK', '2 BHK', '3 BHK', '4 BHK', '5 BHK', '6 BHK', '1 RK', 'Land']:
        new_row[f'building type_{config}'] = features.get(f'building type_{config}')
        new_row[f'area low {config}'] = features.get(f'area low {config}')
        new_row[f'area high {config}'] = features.get(f'area high {config}')
        new_row[f'price low {config}'] = features.get(f'price low {config}')
        new_row[f'price high {config}'] = features.get(f'price high {config}')
    
    data_refined.append(new_row)

df_final_refined_v2 = pd.DataFrame(data_refined).set_index('PropertyName')


In [22]:
df_final_refined_v2['building type_Land'] = df_final_refined_v2['building type_Land'].replace({'':'Land'})

In [23]:
df['PriceDetails'][10]

"{'1 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '550 sq.ft.', 'price-range': '₹ 1.8 - 2.2 Cr'}, '2 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '850 sq.ft.', 'price-range': '₹ 2.8 - 3.3 Cr'}, '3 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,250 sq.ft.', 'price-range': '₹ 4 - 4.8 Cr'}}"

In [24]:
df_final_refined_v2

,building type_1 BHK,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,building type_2 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,...,building type_1 RK,area low 1 RK,area high 1 RK,price low 1 RK,price high 1 RK,building type_Land,area low Land,area high Land,price low Land,price high Land
PropertyName,,,,,,,,,,,,,,,,,,,,,
Lodha Bellissimo,None,NaN,NaN,NaN,NaN,Apartment,1250.0,1250.0,8.0,9.5,...,None,None,None,None,None,None,None,None,None,None
Oberoi Sky City,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
Godrej Platinum,None,NaN,NaN,NaN,NaN,Apartment,1150.0,1150.0,4.2,5.0,...,None,None,None,None,None,None,None,None,None,None
Hiranandani Gardens,None,NaN,NaN,NaN,NaN,Apartment,1350.0,1350.0,5.5,6.5,...,None,None,None,None,None,None,None,None,None,None
Raheja Vivarea,Apartment,650.0,650.0,2.8,3.2,Apartment,950.0,950.0,3.8,4.2,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Concorde Galleria,None,NaN,NaN,NaN,NaN,Apartment,1150.0,1150.0,2.5,3.0,...,None,None,None,None,None,None,None,None,None,None
Purvankara Sky Forest,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
Shapoorji Pallonji Joyville,None,NaN,NaN,NaN,NaN,Apartment,1050.0,1050.0,1.8,2.2,...,None,None,None,None,None,None,None,None,None,None


In [25]:
categorical_columns = df_final_refined_v2.select_dtypes(include=['object']).columns.tolist()

In [26]:
categorical_columns

['building type_1 BHK',
 'building type_2 BHK',
 'building type_3 BHK',
 'building type_4 BHK',
 'building type_5 BHK',
 'building type_6 BHK',
 'area low 6 BHK',
 'area high 6 BHK',
 'price low 6 BHK',
 'price high 6 BHK',
 'building type_1 RK',
 'area low 1 RK',
 'area high 1 RK',
 'price low 1 RK',
 'price high 1 RK',
 'building type_Land',
 'area low Land',
 'area high Land',
 'price low Land',
 'price high Land']

In [27]:
ohe_df = pd.get_dummies(df_final_refined_v2, columns=categorical_columns, drop_first=True)

In [28]:
ohe_df.fillna(0,inplace=True)

In [29]:
ohe_df

,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,area low 3 BHK,area high 3 BHK,price low 3 BHK,price high 3 BHK,area low 4 BHK,area high 4 BHK,price low 4 BHK,price high 4 BHK,area low 5 BHK,area high 5 BHK,price low 5 BHK,price high 5 BHK
PropertyName,,,,,,,,,,,,,,,,,,,,
Lodha Bellissimo,0.0,0.0,0.0,0.0,1250.0,1250.0,8.0,9.5,1850.0,1850.0,12.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Oberoi Sky City,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2100.0,2100.0,14.0,16.0,2850.0,2850.0,19.0,22.0,0.0,0.0,0.0,0.0
Godrej Platinum,0.0,0.0,0.0,0.0,1150.0,1150.0,4.2,5.0,1650.0,1650.0,5.8,6.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hiranandani Gardens,0.0,0.0,0.0,0.0,1350.0,1350.0,5.5,6.5,1850.0,1850.0,7.5,8.5,2450.0,2450.0,10.0,12.0,0.0,0.0,0.0,0.0
Raheja Vivarea,650.0,650.0,2.8,3.2,950.0,950.0,3.8,4.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Concorde Galleria,0.0,0.0,0.0,0.0,1150.0,1150.0,2.5,3.0,1600.0,1600.0,3.5,4.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Purvankara Sky Forest,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1700.0,1700.0,6.2,7.2,2300.0,2300.0,8.5,9.8,0.0,0.0,0.0,0.0
Shapoorji Pallonji Joyville,0.0,0.0,0.0,0.0,1050.0,1050.0,1.8,2.2,1500.0,1500.0,2.5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Apply the scaler to the entire dataframe
ohe_df_normalized = pd.DataFrame(scaler.fit_transform(ohe_df), columns=ohe_df.columns, index=ohe_df.index)



In [31]:
ohe_df_normalized.head()

,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,area low 3 BHK,area high 3 BHK,price low 3 BHK,price high 3 BHK,area low 4 BHK,area high 4 BHK,price low 4 BHK,price high 4 BHK,area low 5 BHK,area high 5 BHK,price low 5 BHK,price high 5 BHK
PropertyName,,,,,,,,,,,,,,,,,,,,
Lodha Bellissimo,-0.456535,-0.456535,-0.438609,-0.441782,0.994276,0.994276,2.136412,2.274313,0.614696,0.614696,1.413582,1.539666,-0.845306,-0.845306,-0.761851,-0.768393,-0.446738,-0.446738,-0.405633,-0.407749
Oberoi Sky City,-0.456535,-0.456535,-0.438609,-0.441782,-1.194142,-1.194142,-1.083061,-1.090681,0.974889,0.974889,1.887640,1.969153,1.310216,1.310216,1.827655,1.913742,-0.446738,-0.446738,-0.405633,-0.407749
Godrej Platinum,-0.456535,-0.456535,-0.438609,-0.441782,0.819203,0.819203,0.607162,0.680368,0.326542,0.326542,-0.055996,-0.070909,-0.845306,-0.845306,-0.761851,-0.768393,-0.446738,-0.446738,-0.405633,-0.407749
Hiranandani Gardens,-0.456535,-0.456535,-0.438609,-0.441782,1.169350,1.169350,1.130327,1.211683,0.614696,0.614696,0.346953,0.358578,1.007687,1.007687,0.601047,0.694589,-0.446738,-0.446738,-0.405633,-0.407749
Raheja Vivarea,1.904137,1.904137,1.745013,1.763890,0.469056,0.469056,0.446189,0.397000,-2.050728,-2.050728,-1.430763,-1.466740,-0.845306,-0.845306,-0.761851,-0.768393,-0.446738,-0.446738,-0.405633,-0.407749


In [32]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_sim2 = cosine_similarity(ohe_df_normalized)

In [33]:
cosine_sim2.shape

(149, 149)

In [34]:
def recommend_properties_with_scores(property_name, top_n=247):
    
    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim2[ohe_df_normalized.index.get_loc(property_name)]))
    
    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
    
    # Retrieve the names of the top properties using the indices
    top_properties = ohe_df_normalized.index[top_indices].tolist()
    
    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })
    
    return recommendations_df

# Test the recommender function using a property name
recommend_properties_with_scores('Piramal Aranya Phase 5')

,PropertyName,SimilarityScore
0,Piramal Aranya Phase 6,0.998909
1,Piramal Aranya Phase 4,0.998602
2,Piramal Aranya Phase 7,0.996157
3,Piramal Aranya Phase 3,0.993652
4,Oberoi Sky City,0.992977
...,...,...
143,Raheja Vivarea,-0.649911
144,Wadhwa Group - Serenity,-0.650357
145,Wadhwa Group - Aerocity,-0.651865
146,Wadhwa Group - Upper Vikhroli,-0.652369


In [35]:
df[['PropertyName','LocationAdvantages']]['LocationAdvantages'].head(20)

0     {'Sea Face': '500 m', 'Haji Ali': '2 km', 'Ban...
1     {'Worli Sea Face': '800 m', 'Bandra-Worli Sea ...
2     {'IIT Bombay': '3 km', 'Eastern Express Highwa...
3     {'Powai Lake': '500 m', 'IIT Bombay': '2 km', ...
4     {'Mumbai Airport': '3 km', 'BKC': '4 km', 'San...
5     {'Upvan Lake': '2 km', 'Korum Mall': '3 km', '...
6     {'Byculla Zoo': '1 km', 'Mumbai Central': '2 k...
7     {'IIT Bombay': '3 km', 'Eastern Express Highwa...
8     {'Mumbai Airport': '2 km', 'BKC': '5 km', 'And...
9     {'Wadala TT': '1 km', 'Eastern Freeway': '500 ...
10    {'Mulund Station': '2 km', 'Eastern Express Hi...
11    {'Eastern Express Highway': '2 km', 'Bhandup S...
12    {'Upvan Lake': '3 km', 'Korum Mall': '4 km', '...
13    {'IIT Bombay': '3 km', 'Eastern Express Highwa...
14    {'Upvan Lake': '2 km', 'Korum Mall': '3 km', '...
15    {'Mulund Station': '2 km', 'Eastern Express Hi...
16    {'Byculla Zoo': '1 km', 'Mumbai Central': '2 k...
17    {'Mumbai Airport': '1.5 km', 'BKC': '4 km'

In [36]:
def distance_to_meters(distance_str):
    try:
        if 'km' in distance_str or 'KM' in distance_str:
            return float(distance_str.split()[0]) * 1000
        elif 'm' in distance_str or 'M' in distance_str:
            return float(distance_str.split()[0])
        else:
            return None
    except:
        return None

In [37]:
# Extract distances for each location
location_matrix = {}
for index, row in df.iterrows():
    distances = {}
    for location, distance in ast.literal_eval(row['LocationAdvantages']).items():
        distances[location] = distance_to_meters(distance)
    location_matrix[index] = distances

# Convert the dictionary to a dataframe
location_df = pd.DataFrame.from_dict(location_matrix, orient='index')

# Display the first few rows
location_df.sample(20)

,Sea Face,Haji Ali,Bandra-Worli Sea Link,Chhatrapati Shivaji Maharaj International Airport,CSMT Railway Station,Worli Sea Face,Mumbai Airport,Prabhadevi Station,Mahalaxmi Station,IIT Bombay,...,Belapur,Goregaon Station,Film City,Aarey Colony,Panvel Station,NAVI Mumbai Airport,Kharghar,Thakur Village,Kharghar Station,Central Park
105,NaN,NaN,NaN,NaN,NaN,NaN,15000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2000.0,NaN,NaN,800.0,14000.0,3000.0,2000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,7000.0,NaN,NaN,2000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,NaN,NaN,NaN,NaN,NaN,NaN,15000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,NaN,NaN,NaN,NaN,NaN,NaN,12000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,NaN,NaN,NaN,NaN,NaN,NaN,15000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,15000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,16000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
location_df.columns[10:50]

Index(['Eastern Express Highway', 'Powai Lake', 'Kanjurmarg Station',
       'Metro Station', 'BKC', 'Santacruz Station', 'Western Express Highway',
       'Upvan Lake', 'Korum Mall', 'Thane Station', 'Mumbai-Pune Expressway',
       'Byculla Zoo', 'Mumbai Central', 'Phoenix Marketcity', 'CSMT Station',
       'Eastern Freeway', 'Andheri Station', 'Wadala TT', 'Sion',
       'Mumbai Port', 'Mulund Station', 'Mulund Lake', 'Thane',
       'Bhandup Station', 'Mulund', 'Versova Beach', 'Lokhandwala',
       'Dombivli Station', 'Kalyan', 'Juhu Beach', 'Santacruz Airport',
       'Linking Road', 'Bandra Station', 'Kurla Station', 'Malad Station',
       'Infiniti Mall', 'Mindspace', 'Bandra Bandstand', 'Carter Road',
       'Chembur Station'],
      dtype='object')

In [39]:
location_df.index = df.PropertyName

In [40]:
location_df.head()

,Sea Face,Haji Ali,Bandra-Worli Sea Link,Chhatrapati Shivaji Maharaj International Airport,CSMT Railway Station,Worli Sea Face,Mumbai Airport,Prabhadevi Station,Mahalaxmi Station,IIT Bombay,...,Belapur,Goregaon Station,Film City,Aarey Colony,Panvel Station,NAVI Mumbai Airport,Kharghar,Thakur Village,Kharghar Station,Central Park
PropertyName,,,,,,,,,,,,,,,,,,,,,
Lodha Bellissimo,500.0,2000.0,1500.0,12000.0,8000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Oberoi Sky City,NaN,NaN,2000.0,NaN,NaN,800.0,14000.0,3000.0,2000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Godrej Platinum,NaN,NaN,NaN,NaN,NaN,NaN,8000.0,NaN,NaN,3000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hiranandani Gardens,NaN,NaN,NaN,NaN,NaN,NaN,7000.0,NaN,NaN,2000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Raheja Vivarea,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
location_df.fillna(54000,inplace=True)

In [42]:
location_df

,Sea Face,Haji Ali,Bandra-Worli Sea Link,Chhatrapati Shivaji Maharaj International Airport,CSMT Railway Station,Worli Sea Face,Mumbai Airport,Prabhadevi Station,Mahalaxmi Station,IIT Bombay,...,Belapur,Goregaon Station,Film City,Aarey Colony,Panvel Station,NAVI Mumbai Airport,Kharghar,Thakur Village,Kharghar Station,Central Park
PropertyName,,,,,,,,,,,,,,,,,,,,,
Lodha Bellissimo,500.0,2000.0,1500.0,12000.0,8000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Oberoi Sky City,54000.0,54000.0,2000.0,54000.0,54000.0,800.0,14000.0,3000.0,2000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Godrej Platinum,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,8000.0,54000.0,54000.0,3000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Hiranandani Gardens,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,7000.0,54000.0,54000.0,2000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Raheja Vivarea,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,3000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Concorde Galleria,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Purvankara Sky Forest,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Shapoorji Pallonji Joyville,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0


In [43]:
from sklearn.preprocessing import StandardScaler
# Initialize the scaler
scaler = StandardScaler()

# Apply the scaler to the entire dataframe
location_df_normalized = pd.DataFrame(scaler.fit_transform(location_df), columns=location_df.columns, index=location_df.index)

In [44]:
location_df_normalized

,Sea Face,Haji Ali,Bandra-Worli Sea Link,Chhatrapati Shivaji Maharaj International Airport,CSMT Railway Station,Worli Sea Face,Mumbai Airport,Prabhadevi Station,Mahalaxmi Station,IIT Bombay,...,Belapur,Goregaon Station,Film City,Aarey Colony,Panvel Station,NAVI Mumbai Airport,Kharghar,Thakur Village,Kharghar Station,Central Park
PropertyName,,,,,,,,,,,,,,,,,,,,,
Lodha Bellissimo,-12.165525,-12.165525,-8.614692,-12.165525,-12.165525,0.082199,1.154965,0.082199,0.082199,0.459723,...,0.143299,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199
Oberoi Sky City,0.082199,0.082199,-8.531537,0.082199,0.082199,-12.165525,-0.620850,-12.165525,-12.165525,0.459723,...,0.143299,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199
Godrej Platinum,0.082199,0.082199,0.116641,0.082199,0.082199,0.082199,-0.887222,0.082199,0.082199,-2.130499,...,0.143299,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199
Hiranandani Gardens,0.082199,0.082199,0.116641,0.082199,0.082199,0.082199,-0.931618,0.082199,0.082199,-2.181288,...,0.143299,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199
Raheja Vivarea,0.082199,0.082199,0.116641,0.082199,0.082199,0.082199,-1.109199,0.082199,0.082199,0.459723,...,0.143299,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Concorde Galleria,0.082199,0.082199,0.116641,0.082199,0.082199,0.082199,1.154965,0.082199,0.082199,0.459723,...,0.143299,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199
Purvankara Sky Forest,0.082199,0.082199,0.116641,0.082199,0.082199,0.082199,1.154965,0.082199,0.082199,0.459723,...,0.143299,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199
Shapoorji Pallonji Joyville,0.082199,0.082199,0.116641,0.082199,0.082199,0.082199,1.154965,0.082199,0.082199,0.459723,...,0.143299,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199


In [45]:
cosine_sim3 = cosine_similarity(location_df_normalized)

In [46]:
cosine_sim3.shape

(149, 149)

In [47]:
def recommend_properties_with_scores(property_name, top_n=247):
    
    cosine_sim_matrix = 0.8*cosine_sim1 + 1*cosine_sim2 + 0.5*cosine_sim3
    # cosine_sim_matrix = cosine_sim3
    
    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim_matrix[location_df_normalized.index.get_loc(property_name)]))
    
    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
    
    # Retrieve the names of the top properties using the indices
    top_properties = location_df_normalized.index[top_indices].tolist()
    
    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })
    
    return recommendations_df

# Test the recommender function using a property name
recommend_properties_with_scores('Oberoi Sky City')

,PropertyName,SimilarityScore
0,Raheja Arthan,1.171490
1,Bharat Diamond Bourse Apartments,1.106776
2,Piramal Aranya Phase 7,1.008133
3,Godrej Ultimate Plus,1.003622
4,Godrej Apex Plus,0.999289
...,...,...
143,Wadhwa Group - Aerocity,-0.679198
144,Wadhwa Group - Paradise,-0.681507
145,Wadhwa Group - Serenity,-0.687467
146,Wadhwa Group - Upper Vikhroli,-0.697429


In [48]:
(3*cosine_sim3 + 5*cosine_sim2 + 6*cosine_sim1).shape

(149, 149)

In [49]:
location_df

,Sea Face,Haji Ali,Bandra-Worli Sea Link,Chhatrapati Shivaji Maharaj International Airport,CSMT Railway Station,Worli Sea Face,Mumbai Airport,Prabhadevi Station,Mahalaxmi Station,IIT Bombay,...,Belapur,Goregaon Station,Film City,Aarey Colony,Panvel Station,NAVI Mumbai Airport,Kharghar,Thakur Village,Kharghar Station,Central Park
PropertyName,,,,,,,,,,,,,,,,,,,,,
Lodha Bellissimo,500.0,2000.0,1500.0,12000.0,8000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Oberoi Sky City,54000.0,54000.0,2000.0,54000.0,54000.0,800.0,14000.0,3000.0,2000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Godrej Platinum,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,8000.0,54000.0,54000.0,3000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Hiranandani Gardens,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,7000.0,54000.0,54000.0,2000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Raheja Vivarea,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,3000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Concorde Galleria,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Purvankara Sky Forest,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Shapoorji Pallonji Joyville,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0


In [51]:
import pickle
pickle.dump(location_df,open('location_distance.pkl','wb'))

In [52]:
pickle.dump(cosine_sim1,open('cosine_sim1.pkl','wb'))
pickle.dump(cosine_sim2,open('cosine_sim2.pkl','wb'))
pickle.dump(cosine_sim3,open('cosine_sim3.pkl','wb'))